In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image

2023-05-18 07:32:19.722658: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-18 07:32:19.999440: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-18 07:32:20.000469: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 07:32:20.924461: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
image_Folder = 'images/'

In [3]:
import os

In [4]:
list_dirs = os.listdir(image_Folder)

In [5]:
print(list_dirs)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [6]:
def get_index(dir):
    if dir == "angry":
        return 0
    elif dir == "disgust":
        return 1
    elif dir == "fear":
        return 2
    elif dir == "happy":
        return 3
    elif dir == "neutral":
        return 4
    elif dir == "sad":
        return 5
    else:
        return 6

def get_emotion(index):
    if index == 0:
        return "angry"
    elif index == 1:
        return "disgust"
    elif index == 2:
        return "fear"
    elif index == 3:
        return "happy"
    elif index == 4:
        return "neutral"
    elif index == 5:
        return "sad"
    else:
        return "surprise"

In [7]:
imgs = []
labels = []

for dir in list_dirs:
    list_images = os.listdir(image_Folder + dir)
    for image in list_images:
        img = np.asarray(Image.open(image_Folder + dir + "/" + image)) 
        img = img.reshape(48, 48, 1) / 255.0
        imgs.append(img)
        labels.append(get_index(dir))
    del(list_images)
    del(dir)

In [8]:
print(len(imgs))
print(len(labels))

25596
25596


In [9]:
imgs = np.asarray(imgs)
labels = np.asarray(labels)

In [10]:
np.unique(labels)

array([0, 1, 2, 3, 4, 5, 6])

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(imgs, labels, test_size=0.2, random_state=42)

In [13]:
del(imgs)

In [14]:
del(labels)

In [15]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(20476, 48, 48, 1)
(20476,)
(5120, 48, 48, 1)
(5120,)


In [16]:
X_train[0].shape

(48, 48, 1)

Model

In [17]:
# mobilenet_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
# # mobilenet_model = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/classification/2"

# pretrained_model = hub.KerasLayer(mobilenet_model,input_shape=(224,224,1),trainable=False)

In [55]:
# model = tf.keras.Sequential([
#     tf.keras.applications.vgg16(input_shape=(224,224,1)),
#     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.Dense(7)
# ])

# from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input, Concatenate
# from keras.applications.vgg16 import VGG16

# img_size_target = 224
# img_input = Input(shape=(img_size_target, img_size_target, 1))
# img_conc = Concatenate()([img_input, img_input, img_input])  
# model = VGG16(input_tensor=img_conc)

# model = tf.keras.Sequential([
#     tf.keras.layers.Flatten(input_shape=(224,224)),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(128,activation='relu'),
#     tf.keras.layers.Dense(256,activation='relu'),
#     tf.keras.layers.Dense(128,activation='relu'),
#     tf.keras.layers.Dense(64,activation='relu'),
#     tf.keras.layers.Dense(7,activation='sigmoid')
# ])

num_of_classes = 7

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(8, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))


model.add(tf.keras.layers.Conv2D(32, kernel_size=(4,4), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))


# model.add(tf.keras.layers.Conv2D(256, kernel_size=(3,3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
# model.add(tf.keras.layers.Dropout(0.25))


model.add(tf.keras.layers.Flatten())

# model.add(tf.keras.layers.Dense(512, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))


model.add(tf.keras.layers.Dense(num_of_classes, activation='sigmoid'))

# model = tf.keras.Sequential([
#     pretrained_model,
#     tf.keras.layers.Dense(7)
# ])

model.compile(optimizer="adam",loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_38 (Conv2D)          (None, 46, 46, 8)         80        
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 23, 23, 8)        0         
 g2D)                                                            
                                                                 
 dropout_78 (Dropout)        (None, 23, 23, 8)         0         
                                                                 
 conv2d_39 (Conv2D)          (None, 20, 20, 32)        4128      
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 10, 10, 32)       0         
 g2D)                                                            
                                                                 
 dropout_79 (Dropout)        (None, 10, 10, 32)      

In [56]:
model.fit(X_train,Y_train,epochs=50)

Epoch 1/50
640/640 [==============================] - 7s 9ms/step - loss: 1.8254 - accuracy: 0.2122
Epoch 2/50
640/640 [==============================] - 6s 9ms/step - loss: 1.6937 - accuracy: 0.3106
Epoch 3/50
640/640 [==============================] - 6s 9ms/step - loss: 1.5870 - accuracy: 0.3727
Epoch 4/50
640/640 [==============================] - 6s 9ms/step - loss: 1.5313 - accuracy: 0.3989
Epoch 5/50
640/640 [==============================] - 6s 9ms/step - loss: 1.4821 - accuracy: 0.4181
Epoch 6/50
640/640 [==============================] - 6s 10ms/step - loss: 1.4537 - accuracy: 0.4304
Epoch 7/50
640/640 [==============================] - 6s 9ms/step - loss: 1.4241 - accuracy: 0.4459
Epoch 8/50
640/640 [==============================] - 6s 9ms/step - loss: 1.4000 - accuracy: 0.4561
Epoch 9/50
640/640 [==============================] - 6s 9ms/step - loss: 1.3755 - accuracy: 0.4682
Epoch 10/50
640/640 [==============================] - 6s 9ms/step - loss: 1.3533 - accuracy: 0.478

In [58]:
model.evaluate(X_test,Y_test)

160/160 [==============================] - 1s 3ms/step - loss: 1.3558 - accuracy: 0.4887


[1.355818510055542, 0.4886718690395355]

In [21]:
model.save("face_expression-v2.h5")